In [1]:
require(tidyverse)
require(data.table)

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.1  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [48]:
score_dir <- '/oak/stanford/groups/mrivas/projects/PRS/private_output/snpnet_biomarker_PRS/biomarkers_20190407/8_score_all'
phe_info_f <- 'Biomarker_PRSs.tsv'
phe_info <- fread(
    cmd=paste('cat ', phe_info_f, ' |sed -e "s/^#//g"'), sep='\t', data.table=F
)

In [49]:
population_f <- '/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_white_british.phe'
population_df <- fread(population_f, head=F, data.table=F, sep='\t')
colnames(population_df) <- c('FID', 'IID')

In [50]:
read_sscore <- function(score_file, GBE_id){
    fread(
        cmd=paste0('cat ', score_file, ' | cut -f2,6', ' |sed -e "s/^#//g"'), 
        sep='\t', data.table=F
    ) %>% 
    mutate(
        GBE_ID = GBE_id
    ) %>%
    rename(
        Score = SCORE1_AVG
    )
}

In [51]:
read_sscore_wrapper <- function(GBE_id, phenotype_info = phe_info, score_d = score_dir){
    score_f_basename <- phenotype_info %>% filter(GBE_ID == GBE_id) %>%  
    select(sscore_file) %>% pull()
    score_f <- file.path(score_d, score_f_basename)
    read_sscore(score_f, GBE_id)
}

In [52]:
read_sscore_all <- function(phenotype_info){
    phenotype_info %>% select(GBE_ID) %>% pull() %>% 
    lapply(read_sscore_wrapper) %>% reduce(bind_rows)
}

In [53]:
scores_all_df <- read_sscore_all(phe_info)

In [57]:
scores_long_df <- scores_all_df %>% 
right_join(
    population_df %>% select(IID), by = 'IID'
)

In [61]:
scores_all_df  %>% dim() %>% print()
scores_long_df %>% dim() %>% print()
scores_all_df  %>% select(IID) %>% unique() %>% dim() %>% print()
scores_long_df %>% select(IID) %>% unique() %>% dim() %>% print()
scores_all_df  %>% select(GBE_ID) %>% unique() %>% dim() %>% print()
scores_long_df %>% select(GBE_ID) %>% unique() %>% dim() %>% print()

[1] 18558326        3
[1] 12811738        3
[1] 488377      1
[1] 337151      1
[1] 38  1
[1] 38  1


In [62]:
337151 * 38

[1] 12811738

In [67]:
scores_wide_df <- scores_long_df %>% spread(GBE_ID, Score, fill=0)


In [68]:
scores_wide_df %>% dim()

[1] 337151     39

In [69]:
scores_wide_df %>% 
fwrite(
    'Biomarkers_snpnet_PRSs.tsv', sep='\t'
)

```
$ gzip -9 Biomarkers_snpnet_PRSs.tsv
```